# Overview map

In [ ]:
import pandas as pd
import geopandas as gpd
import subprocess
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import numpy as np
import sys, os
sys.path.append(os.getcwd() + '/..')
from utils import mkdir, submission_summary, needs_size_filter, size_filter, plot_map, plot_date_hist, maps
import os
import matplotlib.patches as mpatches
from oggm import utils

import seaborn as sns
import cartopy
import cartopy.crs as ccrs
sns.set_context('poster')
sns.set_style('whitegrid')

In [ ]:
# go down from rgi7_scripts/workflow
data_dir = '../../../rgi7_data/'

# Level 2 GLIMS files
l3_dir = os.path.join(data_dir, 'l3_rgi7a')

In [ ]:
rgi = []
for reg in range(1, 20):
    shp = gpd.read_file(utils.get_rgi_region_file(f'{reg:02d}', version='60'))
    shp['area'] = shp.to_crs({'proj':'cea'}).area
    shp['geometry'] = shp.representative_point()
    shp['O1Region'] = f'{reg:02d}'
    rgi.append(shp)
rgi6 = pd.concat(rgi)
rgi6['area'] = rgi6['area'] * 1e-6

In [ ]:
rgi = []
for reg in range(1, 20):
    shp = gpd.read_file(os.path.join(l3_dir, f'RGI{reg:02d}', f'RGI{reg:02d}.shp'))
    shp['geometry'] = shp.representative_point()
    shp['O1Region'] = f'{reg:02d}'
    rgi.append(shp)
rgi7 = pd.concat(rgi)
rgi7['area'] = rgi7['area'] * 1e-6

In [ ]:
rgi6 = rgi6.loc[rgi6['Connect'] != 2]

In [ ]:
rgi6['Area'].sum(), rgi6['area'].sum(), rgi7['area'].sum()

In [ ]:
df = rgi6.groupby('O1Region').sum()[['area']]
df.columns = ['Area RGI6']

In [ ]:
df['Area RGI7'] = rgi7.groupby('O1Region').sum()[['area']] 

In [ ]:
df['Diff A (%)'] = (df['Area RGI7'] / df['Area RGI6'] - 1) * 100

In [ ]:
df['N RGI6'] = rgi6.groupby('O1Region').count()['area']
df['N RGI7'] = rgi7.groupby('O1Region').count()['area']

df['Diff N (%)'] = (df['N RGI7'] / df['N RGI6'] - 1) * 100

In [ ]:
ss = df.sum()
ss.name = 'Global'
df = df.append(ss)

df['Diff A (%)'] = (df['Area RGI7'] / df['Area RGI6'] - 1) * 100
df['Diff N (%)'] = (df['N RGI7'] / df['N RGI6'] - 1) * 100

In [ ]:
df['Area RGI6'] = df['Area RGI6'].astype(int) 
df['Area RGI7'] = df['Area RGI7'].astype(int) 
df['N RGI6'] = df['N RGI6'].astype(int) 
df['N RGI7'] = df['N RGI7'].astype(int) 

df['Diff A (%)'] = df['Diff A (%)'].round(1).replace(-0, 0)
df['Diff N (%)'] = df['Diff N (%)'].round(1).replace(-0, 0)

In [ ]:
df

In [ ]:
print(df.to_markdown())

In [ ]:
len(rgi)

In [ ]:
import salem
df = salem.read_shapefile(os.path.join(data_dir, '00_rgi70_regions', '00_rgi70_O1Regions', '00_rgi70_O1Regions.shp'))

In [ ]:
f = plt.figure(figsize=(12, 7))
ax = plt.axes(projection=ccrs.EqualEarth())
ax.stock_img()
# ax.add_feature(cartopy.feature.COASTLINE);
# ax.coastlines('50m')

handles = []

df.plot(ax=ax, transform=ccrs.PlateCarree(), edgecolor='k', facecolor='w', alpha=0.3);

s = rgi7.loc[rgi7.is_rgi6 == 1]
ax.scatter(s.CenLon.values, s.CenLat.values, color='C0', s=1, transform=ccrs.PlateCarree());
handles.append(mpatches.Patch(facecolor='C0', label=f"Same as RGI6"))  #  - N={len(s)}, A={int(s['area'].sum()*1e-6)} km²


s = rgi7.loc[rgi7.is_rgi6 == 0]
ax.scatter(s.CenLon.values, s.CenLat.values, color='C3', s=1, transform=ccrs.PlateCarree());
handles.append(mpatches.Patch(facecolor='C3', label=f"New in RGI7"))

df.plot(ax=ax, transform=ccrs.PlateCarree(), edgecolor='k', facecolor='none', linewidth=2);

did1 = False
did10 = False
for i, r in df.iterrows():
    tx = r.min_x + 4
    ty = r.min_y 
    fs = 12
    t = int(r.o1region)
    
    if t == 1:
        tx = r.min_x + 35
        if did1:
            continue
        did1 = True
    if t == 3:
        tx = r.min_x + 20
        ty = r.min_y - 1
    if t == 5:
        tx = r.min_x + 21
    if t == 6:
        ty = r.min_y - 7.
    if t == 8:
        tx = r.min_x + 24
    if t == 9:
        tx = r.min_x 
    if t == 11:
        tx = r.min_x - 8.5
    if t == 10:
        tx = r.min_x + 1
        ty = r.min_y + 16
        if not did10:
            did10 = True
            continue
    if t == 13:
        ty = r.min_y + 12
        tx = r.min_x + 30
    if t in [14, 12, 16, 17]:
        tx = r.min_x 
    if t == 15:
        tx = r.min_x + 30
    if t == 17:
        ty = r.min_y + 25
    if t == 18:
        ty = r.min_y + 10
    if t == 19:
        ty = r.min_y + 10
        
    ax.text(tx, ty, t, transform=ccrs.PlateCarree(), 
            ha='left', va='bottom', fontsize=fs)


ax.legend(handles=handles, loc='center left', fontsize=fs);
    
plt.tight_layout();

plt.savefig('../../../rgi7_data/l3_rgi7a_plots/global_map.png', dpi=150, bbox_inches='tight')

# ax.scatter(dfr.lon.values, dfr.lat.values, edgecolor='k', facecolor='tab:cyan', s=24)

# df.plot(ax=ax, transform=ccrs.PlateCarree(), edgecolor='k', facecolor='w', alpha=0.3);
# df.plot(ax=ax, transform=ccrs.PlateCarree(), edgecolor='k', facecolor='none', linewidth=2);

In [ ]:
s1 = rgi7.loc[rgi7.is_rgi6 == 1]
print(f"Same as RGI6: N={len(s1)}, A={int(s1['area'].sum()*1e-6)} km²")

s2 = rgi7.loc[rgi7.is_rgi6 == 0]
print(f"New  in RGI7: N={len(s2)}, A={int(s2['area'].sum()*1e-6)} km²")

print(f"New in %: {s2['area'].sum() / rgi7['area'].sum()}")

print(f"New in %: {len(s2) / len(rgi7)}")

In [ ]:
rgi7.loc[rgi7['area'] == rgi7['area'].min()]

In [ ]:
adf = pd.DataFrame()
adf.loc['< 1 km²', 'RGI6 (N)'] = (rgi6['area'] < 1).sum()
adf.loc['< 1 km²', 'RGI6 (%)'] = (rgi6['area'] < 1).sum() / len(rgi6) * 100
adf.loc['< 1 km²', 'RGI7 (N)'] = (rgi7['area'] < 1).sum()
adf.loc['< 1 km²', 'RGI7 (%)'] = (rgi7['area'] < 1).sum() / len(rgi7) * 100
adf.loc['1-10 km²', 'RGI6 (N)'] = ((rgi6['area'] >= 1) & (rgi6['area'] < 10)).sum()
adf.loc['1-10 km²', 'RGI6 (%)'] = ((rgi6['area'] >= 1) & (rgi6['area'] < 10)).sum() / len(rgi6) * 100
adf.loc['1-10 km²', 'RGI7 (N)'] = ((rgi7['area'] >= 1) & (rgi7['area'] < 10)).sum()
adf.loc['1-10 km²', 'RGI7 (%)'] = ((rgi7['area'] >= 1) & (rgi7['area'] < 10)).sum() / len(rgi7) * 100
adf.loc['10-100 km²', 'RGI6 (N)'] = ((rgi6['area'] >= 10) & (rgi6['area'] < 100)).sum()
adf.loc['10-100 km²', 'RGI6 (%)'] = ((rgi6['area'] >= 10) & (rgi6['area'] < 100)).sum() / len(rgi6) * 100
adf.loc['10-100 km²', 'RGI7 (N)'] = ((rgi7['area'] >= 10) & (rgi7['area'] < 100)).sum()
adf.loc['10-100 km²', 'RGI7 (%)'] = ((rgi7['area'] >= 10) & (rgi7['area'] < 100)).sum() / len(rgi7) * 100
adf.loc['> 100 km²', 'RGI6 (N)'] = (rgi6['area'] >= 100).sum()
adf.loc['> 100 km²', 'RGI6 (%)'] = (rgi6['area'] >= 100).sum() / len(rgi6) * 100
adf.loc['> 100 km²', 'RGI7 (N)'] = (rgi7['area'] >= 100).sum()
adf.loc['> 100 km²', 'RGI7 (%)'] = (rgi7['area'] >= 100).sum() / len(rgi7) * 100
adf.loc['Total'] = adf.sum()
adf[['RGI6 (N)', 'RGI7 (N)']] = adf[['RGI6 (N)', 'RGI7 (N)']].astype(int)
adf[['RGI6 (%)', 'RGI7 (%)']] = adf[['RGI6 (%)', 'RGI7 (%)']].round(2)
adf

In [ ]:
print(adf.to_markdown())

In [ ]:
sns.set_context('talk')
sns.set_style('ticks')

In [ ]:
bins = np.logspace(-2, 3, 100)

h7, b7 = np.histogram(rgi7['area'], bins=bins)
h6, b6 = np.histogram(rgi6['area'], bins=bins)

f, ax = plt.subplots(figsize=(12, 7))
ax.plot(b6[:-1], h6, label='RGI6');
ax.plot(b7[:-1], h7, label='RGI7');
ax.set_xscale('log')
ax.set_yscale('log')
plt.legend()
ax.set_title('Number of glaciers per size category')
ax.set_xlabel('Glacier area (km², logscale)');
ax.set_ylabel('Glacier number (logscale)');
plt.savefig('../../../rgi7_data/l3_rgi7a_plots/global_histogram.png', dpi=150, bbox_inches='tight')

In [ ]:
bins = np.logspace(-2, 2, 100)
reg = ['13', '14', '15']

h7, b7 = np.histogram(rgi7.loc[rgi7.O1Region.isin(reg)]['area'], bins=bins)
h6, b6 = np.histogram(rgi6.loc[rgi6.O1Region.isin(reg)]['area'], bins=bins)

f, ax = plt.subplots(figsize=(12, 7))
ax.plot(b6[:-1], h6, label='RGI6');
ax.plot(b7[:-1], h7, label='RGI7');
ax.set_xscale('log')
ax.set_yscale('log')
plt.legend()
ax.set_title(f'Number of glaciers per size category (region {reg})')
ax.set_xlabel('Glacier area (km², logscale)');
ax.set_ylabel('Glacier number (logscale)');

In [ ]:
rgi7['year'] = [int(y.split('-')[0]) for y in rgi7['src_date']]

In [ ]:
rgi6['year'] = [int(y[0:4]) for y in rgi6['BgnDate']]
rgi6['year'].unique()

In [ ]:
(rgi6['year'] == -999).sum()

In [ ]:
dy6 = np.abs(rgi6['year'] - 2000)
dy7 = np.abs(rgi7['year'] - 2000)

In [ ]:
ydf = pd.DataFrame()
ydf.loc['2000 ± 2 years', 'RGI6 (%)'] = (dy6 <= 2).sum() / len(rgi6) * 100
ydf.loc['2000 ± 2 years', 'RGI7 (%)'] = (dy7 <= 2).sum() / len(rgi7) * 100
ydf.loc['2000 ± 5 years', 'RGI6 (%)'] = ((dy6 <= 5) & (dy6 > 2)).sum() / len(rgi6) * 100
ydf.loc['2000 ± 5 years', 'RGI7 (%)'] = ((dy7 <= 5) & (dy7 > 2)).sum() / len(rgi7) * 100
ydf.loc['2000 ± 10 years', 'RGI6 (%)'] = ((dy6 <= 10) & (dy6 > 5)).sum() / len(rgi6) * 100
ydf.loc['2000 ± 10 years', 'RGI7 (%)'] = ((dy7 <= 10) & (dy7 > 5)).sum() / len(rgi7) * 100
ydf.loc['> 2000 ± 10 years', 'RGI6 (%)'] = (dy6 > 10).sum() / len(rgi6) * 100
ydf.loc['> 2000 ± 10 years', 'RGI7 (%)'] = (dy7 > 10).sum() / len(rgi7) * 100
ydf = ydf.round(1)
ydf

In [ ]:
print(ydf.to_markdown())

In [ ]:
len(rgi7) - len(rgi7.anlys_id.unique())